In [27]:
# 1. Import Data
import pandas as pd


In [28]:
anime = pd.read_csv('anime/anime.csv')
anime.head(5)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [29]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [30]:
anime.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [31]:
anime_dup = anime[anime.duplicated()]
print(anime_dup)

Empty DataFrame
Columns: [anime_id, name, genre, type, episodes, rating, members]
Index: []


In [32]:
type_values = anime['type'].value_counts()
print(type_values)

TV         3787
OVA        3311
Movie      2348
Special    1676
ONA         659
Music       488
Name: type, dtype: int64


In [33]:
# 2. Simple Recommendation System
m = anime['members'].quantile(0.75)
print(m)

9437.0


In [34]:
qualified_anime = anime.copy().loc[anime['members']>m]
C = anime['rating'].mean()
def WR(x,C=C, m=m):
    v = x['members']
    R = x['rating']
    return (v/(v+m)*R)+(m/(v+m)*C)
qualified_anime['score'] = WR(qualified_anime)
qualified_anime.sort_values('score', ascending =False)
qualified_anime.head(15)

,anime_id,name,genre,type,episodes,rating,members,score
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,9.239896
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,9.227261
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,9.038211
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,9.132749
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,9.002264
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351,8.904307
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855,9.072417
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679,8.833946
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534,8.797668
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109,8.835257


In [35]:
# 3. Genre Based Recommendation System
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(lowercase=True, stop_words = 'english')
anime['genre'] = anime['genre'].fillna('')
tf_idf_matrix = tf_idf.fit_transform(anime['genre'])
tf_idf_matrix.shape

(12294, 46)

In [36]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tf_idf_matrix, tf_idf_matrix)
indices = pd.Series(anime.index, index=anime['name'])
indices = indices.drop_duplicates()
def recommendations (name, cosine_sim = cosine_sim):
    similarity_scores = list(enumerate(cosine_sim[indices[name]]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:21]
    anime_indices = [i[0] for i in similarity_scores]
    return anime['name'].iloc[anime_indices]
recommendations('Kimi no Na wa.')

5805                           Wind: A Breath of Heart OVA
6394                          Wind: A Breath of Heart (TV)
1111                 Aura: Maryuuin Kouga Saigo no Tatakai
1201                        Angel Beats!: Another Epilogue
1494                                              Harmonie
878                          Shakugan no Shana II (Second)
986                                      Shakugan no Shana
1604                                   Shakugan no Shana S
1959                                             Air Movie
4514                                          Touka Gettan
5031                                        Mizuiro (2003)
5127                                    Venus Versus Virus
4219                           Rokujouma no Shinryakusha!?
11061                                         Renai Boukun
208                          Kokoro ga Sakebitagatterunda.
504      Clannad: After Story - Mou Hitotsu no Sekai, K...
1435                                            True Tea

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
count = CountVectorizer(stop_words = 'english')
count_matrix = count.fit_transform(anime['genre'])
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
recommendations('Kimi no Na wa.', cosine_sim2)

5805                          Wind: A Breath of Heart OVA
6394                         Wind: A Breath of Heart (TV)
1111                Aura: Maryuuin Kouga Saigo no Tatakai
208                         Kokoro ga Sakebitagatterunda.
504     Clannad: After Story - Mou Hitotsu no Sekai, K...
1201                       Angel Beats!: Another Epilogue
1435                                           True Tears
1436                   &quot;Bungaku Shoujo&quot; Memoire
1494                                             Harmonie
1631                                  Kimikiss Pure Rouge
1907                                     Myself; Yourself
1959                                            Air Movie
2300                           Koi to Senkyo to Chocolate
3297                   Koi to Senkyo to Chocolate Special
3530         Otome wa Boku ni Koishiteru: Futari no Elder
3908     Koi to Senkyo to Chocolate: Ikenai Hazuki-sensei
3914                           Myself ; Yourself Specials
4514          

In [38]:
cosine_sim2 = linear_kernel(count_matrix, count_matrix)
recommendations('Kimi no Na wa.', cosine_sim2)

180                  Little Busters!: Refrain
223                                   Clannad
320              Kokoro Connect: Michi Random
518                            Kokoro Connect
613                       Little Busters!: EX
878             Shakugan no Shana II (Second)
986                         Shakugan no Shana
1111    Aura: Maryuuin Kouga Saigo no Tatakai
1604                      Shakugan no Shana S
4363                          Honoo no Mirage
5805              Wind: A Breath of Heart OVA
6394             Wind: A Breath of Heart (TV)
10                       Clannad: After Story
16                    Shigatsu wa Kimi no Uso
25              Suzumiya Haruhi no Shoushitsu
45             Kara no Kyoukai 5: Mujun Rasen
60                         Hotarubi no Mori e
88                                 Cross Game
114                      Sakamichi no Apollon
151                                   Berserk
Name: name, dtype: object